In [ ]:
#Test: Token swap using Aggregator

In [ ]:
import requests
import json
import time
from web3 import Web3

In [ ]:
walletAddress = 'mywalletaddress'
wpk = 'myprivatekey'

# Setup Ropsten provider
w3 = Web3(Web3.HTTPProvider('https://ropsten.infura.io/v3/'))

# Get the nonce to prevent from sending the transaction twice
nonce = w3.eth.getTransactionCount(walletAddress)
print('Nonce:', nonce)

# Store the generic ERC20 abi for later use
ABI = json.load(open('abi.json'))

# ROPSTEN ADDRESSES
WETH = Web3.toChecksumAddress('0xc778417e063141139fce010982780140aa0cd5ab')
DAI = Web3.toChecksumAddress('0xad6d458402f60fd3bd25163575031acdce07538d')
USDC = Web3.toChecksumAddress('0x07865c6e87b9f70255377e024ace6630c1eaa37f')


def calcAmount(initialAmount, contractAddress):
    contract = w3.eth.contract(address=contractAddress, abi=ABI)
    decimals = contract.functions.decimals().call()
    amount = int(initialAmount * 10**decimals)
    return amount
    print(decimals, amount)


def getTokenSymbol(contractAddress):
    contract = w3.eth.contract(address=contractAddress, abi=ABI)
    tokenSymbol = contract.functions.symbol().call()
    return tokenSymbol
    print('Symbol:', tokenSymbol)


In [ ]:
# Define query parameters
initialAmount = 100
slippage = '0.01'  # 0.01 = 1% slippage
sellAmount = calcAmount(initialAmount, USDC)
print('Sell:', sellAmount)


def approve(token, spenderAddress, walletAddress, wpk):

    token = Web3.toChecksumAddress(token)

    contract = w3.eth.contract(address=token, abi=ABI)

    spender = Web3.toChecksumAddress(spenderAddress)
    max_amount = Web3.toWei(2**64 - 1, 'ether')
    nonce = w3.eth.getTransactionCount(walletAddress)

    tx = contract.functions.approve(spender, max_amount).buildTransaction({
        'from': walletAddress,
        'nonce': nonce
    })

    signed_tx = w3.eth.account.signTransaction(tx, wpk)
    tx_hash = w3.eth.sendRawTransaction(signed_tx.rawTransaction)

    print("Approve", w3.toHex(tx_hash))

    w3.eth.wait_for_transaction_receipt(tx_hash)


def getQuote(buyToken, slippage, sellToken, sellAmount):
    # Make the query https://ropsten.api.0x.org/
    query = 'https://ropsten.api.0x.org/swap/v1/quote?buyToken={}&slippage={}&sellToken={}&sellAmount={}'.format(buyToken, slippage, sellToken, sellAmount)
    # Get the query and store the response in data
    response = requests.get(query)
    data = response.json()
    # Print a prettiefied version of the response (optional)
    print(json.dumps(data, indent=2))
    # Convert the buyAmmount to integer
    buyAmount = int(data['buyAmount'])
    # Convert the buyAmount to a readable number
    finalPrice = calcAmount(buyAmount, buyToken)
    # Get the symbol of the 2 tokens
    buyTokenSymbol = getTokenSymbol(buyToken)
    sellTokenSymbol = getTokenSymbol(sellToken)
    print(sellTokenSymbol, '->', buyTokenSymbol)

    approve(data['sellTokenAddress'], data['allowanceTarget'], walletAddress, wpk)

    tx = {
        "from": walletAddress,
        "to": Web3.toChecksumAddress(data['to']),
        "data": data['data'],
        "value": 0,
        "gas": int(data['gas']),
        "gasPrice": int(data['gasPrice']),
        "nonce": nonce + 1,
        "chainId": data['chainId']
    }

    # sign the transaction
    signed_tx = w3.eth.account.sign_transaction(tx, wpk)
    #
    # send transaction
    tx_hash = w3.eth.send_raw_transaction(signed_tx.rawTransaction)
    #
    # get transaction hash
    print(w3.toHex(tx_hash))


In [ ]:
# Use only for testing the query
getQuote(DAI, slippage, USDC, sellAmount)